In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("Final_review.csv")
df_b = pd.read_csv("Final_restuarants.csv")

In [4]:
df.head()

,Unnamed: 0,user_id,business_id,stars,name
0,0,FIk4lQQu1eTe2EpzQ4xhBA,8mIrX_LrOnAqWsB5JrOojQ,4,Pinball Hall Of Fame
1,1,EfnzVzqWJUVEv8ft3fW2sQ,8mIrX_LrOnAqWsB5JrOojQ,4,Pinball Hall Of Fame
2,2,LM3neYWMOghn2bgvYNRi7g,8mIrX_LrOnAqWsB5JrOojQ,5,Pinball Hall Of Fame
3,3,UEKG996TTiwonMpXwH-ixQ,8mIrX_LrOnAqWsB5JrOojQ,3,Pinball Hall Of Fame
4,4,yMh4OF_Li2gJzE5rJ8oCbA,8mIrX_LrOnAqWsB5JrOojQ,5,Pinball Hall Of Fame


In [5]:
df_b.head()

,Unnamed: 0,business_id,name,address,categories
0,0,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,"1775 E Tropicana Ave, Ste 29","Restaurants, Italian"
1,1,tstimHoMcYbkSC4eBA1wEg,Maria's Mexican Restaurant & Bakery,6055 E Lake Mead Blvd,"Mexican, Restaurants, Patisserie/Cake Shop, Fo..."
2,2,kANF0dbeoW34s2vwh6Umfw,Dairy Queen,6125 Spring Mountain Rd,"Fast Food, Food, Restaurants, Ice Cream & Froz..."
3,3,X8mtoSxY8whtmbDlj0D3Aw,Imperial Asian Buffet,4343 N Rancho Dr,"Restaurants, Chinese"
4,4,bJP4l_BGq2CudEu0m-wNjg,Artisan Fine Dining Room,"Artisan Hotel, 1501 W Sahara Ave","Restaurants, Pizza, Italian, American (New)"


## Every Restaruant has an avg users rating of 2.85

In [6]:
df.groupby('user_id')['stars'].count().mean()

2.8562895275820934

In [8]:
ratings = 'stars'
users = 'user_id'
items = 'name'

ratings = np.array(df[ratings])
users = np.array(df[users])
items = np.array(df[items])

In [13]:
from sklearn.preprocessing import LabelEncoder

# heavy lifting encoders
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

# preparation for the csr matrix
u = user_encoder.fit_transform(users)
i = item_encoder.fit_transform(items)
lu = len(np.unique(u))
li = len(np.unique(i))

In [11]:
class InteractionMachine:
    def __init__(self):
        self.user_encoder = LabelEncoder()
        self.item_encoder = LabelEncoder()

    def __repr__(self):
        return 'InteractionMachine()'

    def build(self, users, items, ratings):
        u = self.user_encoder.fit_transform(users)
        i = self.item_encoder.fit_transform(items)
        self.n_users = len(np.unique(u))
        self.n_items = len(np.unique(i))
        self.interactions = csr_matrix((ratings, (u, i)), shape=(self.n_users, self.n_items))
        return self

In [14]:
im = InteractionMachine()

im.build(df['user_id'], df['name'], df['stars'])

interactions = im.interactions

In [15]:
interactions.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [17]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k

In [9]:
from scipy.sparse import csr_matrix

In [10]:
# do this
from lightfm.cross_validation import random_train_test_split

C:\Users\skyli\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [19]:
train, test = random_train_test_split(interactions, test_percentage=0.2)

In [20]:
train

<444310x4911 sparse matrix of type '<class 'numpy.int64'>'
	with 977984 stored elements in COOrdinate format>

In [21]:
test

<444310x4911 sparse matrix of type '<class 'numpy.int64'>'
	with 244496 stored elements in COOrdinate format>

In [22]:
model = LightFM()
model.fit(train, epochs=50)

In [32]:
from lightfm.evaluation import auc_score, precision_at_k

In [33]:
auc_score(model, test).mean()

0.8698465

In [25]:
df.sample(n=2)

,Unnamed: 0,user_id,business_id,stars,name
92035,92035,fJhiFIF_nRVVVv14FEIneg,2HxaDXZmAOH6LiuzGgNdPA,5,Farmer Boys
378480,378480,dEslnkrXM8daKhWwgvbQHQ,mElsJdTjWv9a5b6Ti8L9Pw,5,Great Harvest Bread Co


In [26]:
df[df['user_id'] == 'fJhiFIF_nRVVVv14FEIneg']

,Unnamed: 0,user_id,business_id,stars,name
64268,64268,fJhiFIF_nRVVVv14FEIneg,g8oxQ1Ji9mr9PMjhEfaWSg,4,Dick's Last Resort
66683,66683,fJhiFIF_nRVVVv14FEIneg,rTZVcJZ8G80i6NjLw0SW8Q,1,Silverton Casino
92035,92035,fJhiFIF_nRVVVv14FEIneg,2HxaDXZmAOH6LiuzGgNdPA,5,Farmer Boys
232359,232359,fJhiFIF_nRVVVv14FEIneg,0gWg-kqRLEQbhui8b_v2Xw,2,Hi Scores - Blue Diamond
258435,258435,fJhiFIF_nRVVVv14FEIneg,zEaGcSVPDQipnRdEZp-F6g,5,Dom DeMarco's Pizzeria & Bar
313668,313668,fJhiFIF_nRVVVv14FEIneg,gHq85iAANw1eIOVzkzRrpg,5,Metro Diner
469872,469872,fJhiFIF_nRVVVv14FEIneg,1Z5S_Lkcl6vA3ZvrAG91TA,5,Sofrito Rico Authentic Puerto Rican Cuisine
872744,872744,fJhiFIF_nRVVVv14FEIneg,gkxcnkbrQfEWqyiOah0yoA,5,Mama Bird
1023665,1023665,fJhiFIF_nRVVVv14FEIneg,IOEGLxXwCNiq4P-U359D7Q,5,Those Guys Pies


In [27]:
one_user = 'fJhiFIF_nRVVVv14FEIneg'

In [29]:
user_num = im.user_encoder.transform([one_user])[0]

In [30]:
preds = model.predict(user_num, list(range(im.n_items)))
preds = pd.DataFrame(zip(preds, im.item_encoder.classes_), columns=['pred', 'name'])
preds = preds.sort_values('pred', ascending=False)
preds.head()

,pred,name
1771,8.219633,Hash House A Go Go
280,8.006054,Bacchanal Buffet
2782,7.975440,Mon Ami Gabi
4781,7.788392,Wicked Spoon
1645,7.588023,Gordon Ramsay BurGR


In [34]:
from copy import deepcopy

model = LightFM(loss='warp')

count = 0
best = 0
scores = []
for e in range(100):
    if count > 5: # patience
        break
    model.fit_partial(train, epochs=10)
    auc_train = auc_score(model, train).mean()
    auc_test = auc_score(model, test).mean()
    print(f'Epoch: {e}, Train AUC={auc_train:.3f}, Test AUC={auc_test:.3f}')
    scores.append((auc_train, auc_test))
    if auc_test > best:
        best_model = deepcopy(model)
        best = auc_test
    else:
        count += 1

model = deepcopy(best_model)

Epoch: 0, Train AUC=0.975, Test AUC=0.869
Epoch: 1, Train AUC=0.985, Test AUC=0.861
Epoch: 2, Train AUC=0.989, Test AUC=0.857
Epoch: 3, Train AUC=0.990, Test AUC=0.855
Epoch: 4, Train AUC=0.991, Test AUC=0.854
Epoch: 5, Train AUC=0.992, Test AUC=0.853
Epoch: 6, Train AUC=0.993, Test AUC=0.853


In [35]:
preds = model.predict(user_num, list(range(im.n_items)))
preds = pd.DataFrame(zip(preds, im.item_encoder.classes_), columns=['pred', 'name'])
preds = preds.sort_values('pred', ascending=False)
preds.head()

,pred,name
2819,1.770625,Mr Mamas
4843,1.676623,Yard House
3871,1.651195,SkinnyFATS
432,1.566731,Black Bear Diner
3026,1.493256,Omelet House


In [40]:
df[df['name'].str.contains('India')]

,Unnamed: 0,user_id,business_id,stars,name
14083,14083,DoRCeCcJbrsM2BiAKj3trA,SU56w479vUfFHsvmvQIf7A,4,India Oven
14084,14084,Ct6qLGX16mC08q1GzHUghw,SU56w479vUfFHsvmvQIf7A,1,India Oven
14085,14085,HLn1dQypq_fg8xJ3hUVjEg,SU56w479vUfFHsvmvQIf7A,2,India Oven
14086,14086,_Ryxp7HIcV1cBaUeCptS_A,SU56w479vUfFHsvmvQIf7A,5,India Oven
14087,14087,86ij_78wolDsWWwAB1gGFQ,SU56w479vUfFHsvmvQIf7A,3,India Oven
...,...,...,...,...,...
1268800,1268800,PvUnXsUf52tQMvyd8FCZ4w,mUk-0jToBuzk4KaDXp-uew,5,Taza Indian Kitchen
1268801,1268801,ECY7iCCT-rWua69IbvlrYA,mUk-0jToBuzk4KaDXp-uew,2,Taza Indian Kitchen
1268802,1268802,AwotTxzd_M5ZevluUUVkwQ,mUk-0jToBuzk4KaDXp-uew,3,Taza Indian Kitchen
1268803,1268803,0fG7akW_GqoA3iWMD8_wNg,mUk-0jToBuzk4KaDXp-uew,3,Taza Indian Kitchen


In [45]:
df[df['user_id'] == 'YwaKGmRNnSa3R3N4Hf9jLw']

,Unnamed: 0,user_id,business_id,stars,name
494,494,YwaKGmRNnSa3R3N4Hf9jLw,8mIrX_LrOnAqWsB5JrOojQ,5,Pinball Hall Of Fame
25308,25308,YwaKGmRNnSa3R3N4Hf9jLw,Vs7gc9EE3k9wARuUcN9piA,4,Pan Asian
30272,30272,YwaKGmRNnSa3R3N4Hf9jLw,SAfuIUZMXrOnb3LWyvYjDA,4,Jun's Korean Restaurant
34661,34661,YwaKGmRNnSa3R3N4Hf9jLw,7EZ4Eu7YJ1ltRCC5jXFJrQ,3,Corner Market Buffet
45329,45329,YwaKGmRNnSa3R3N4Hf9jLw,I6qFvR-jVDGqDD_iVG38ng,4,Jenni Pho Grill
...,...,...,...,...,...
1245027,1245027,YwaKGmRNnSa3R3N4Hf9jLw,qkPiymXn9J8gF1_4s9nYbg,3,Feast Buffet
1245575,1245575,YwaKGmRNnSa3R3N4Hf9jLw,BOMgSyi4-TYAnjVn3m0z3A,5,24 Seven Cafe
1257326,1257326,YwaKGmRNnSa3R3N4Hf9jLw,UAMTM6LMA2on079oBMFBPQ,4,Lazy Joe's Fish & Chips
1263861,1263861,YwaKGmRNnSa3R3N4Hf9jLw,jJZwQj3Pwv1OMZ8bQpKxnw,4,PhilHouse


In [46]:
one_user = 'YwaKGmRNnSa3R3N4Hf9jLw'
user_num = im.user_encoder.transform([one_user])[0]
preds = model.predict(user_num, list(range(im.n_items)))
preds = pd.DataFrame(zip(preds, im.item_encoder.classes_), columns=['pred', 'name'])
preds = preds.sort_values('pred', ascending=False)
preds.head()

,pred,name
4056,0.655391,Sura BBQ Buffet
2785,0.620406,Monta Ramen
1852,0.504938,Hot N Juicy Crawfish
4767,0.420770,Wendy's Noodle Cafe
3465,0.408695,Ramen Sora


In [48]:
one_user = '0fG7akW_GqoA3iWMD8_wNg'
user_num = im.user_encoder.transform([one_user])[0]
preds = model.predict(user_num, list(range(im.n_items)))
preds = pd.DataFrame(zip(preds, im.item_encoder.classes_), columns=['pred', 'name'])
preds = preds.sort_values('pred', ascending=False)
preds.head()

,pred,name
281,2.370500,Bachi Burger
1436,2.154949,Firefly
3225,2.019377,Pho Kim Long
1708,1.863677,Grimaldi's Pizzeria
278,1.831104,BabyStacks Cafe


In [49]:
df_b.head()

,Unnamed: 0,business_id,name,address,categories
0,0,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,"1775 E Tropicana Ave, Ste 29","Restaurants, Italian"
1,1,tstimHoMcYbkSC4eBA1wEg,Maria's Mexican Restaurant & Bakery,6055 E Lake Mead Blvd,"Mexican, Restaurants, Patisserie/Cake Shop, Fo..."
2,2,kANF0dbeoW34s2vwh6Umfw,Dairy Queen,6125 Spring Mountain Rd,"Fast Food, Food, Restaurants, Ice Cream & Froz..."
3,3,X8mtoSxY8whtmbDlj0D3Aw,Imperial Asian Buffet,4343 N Rancho Dr,"Restaurants, Chinese"
4,4,bJP4l_BGq2CudEu0m-wNjg,Artisan Fine Dining Room,"Artisan Hotel, 1501 W Sahara Ave","Restaurants, Pizza, Italian, American (New)"


In [72]:
category = []
for i, x in df_b.iterrows():
    lis = x['categories'].split(', ')
    for j in lis:
        if any(substring in j for substring in ['Restaurants','Food']):
            pass
        else:
            value = j
            break
    if value:
        value.strip()
        category.append(value)
    else:
        category = 'None'

In [73]:
category

['Italian',
 'Mexican',
 'Ice Cream & Frozen Yogurt',
 'Chinese',
 'Pizza',
 'Pizza',
 'French',
 'Buffets',
 'Bars',
 'Sandwiches',
 'Middle Eastern',
 'Dive Bars',
 'Delis',
 'American (Traditional)',
 'Sandwiches',
 'Steakhouses',
 'Seafood',
 'Bakeries',
 'Pizza',
 'Mexican',
 'Coffee & Tea',
 'Pizza',
 'Chinese',
 'Japanese',
 'Chinese',
 'Thai',
 'Mexican',
 'Tapas/Small Plates',
 'Mexican',
 'Delis',
 'Southern',
 'American (Traditional)',
 'American (Traditional)',
 'Pizza',
 'American (Traditional)',
 'Japanese',
 'Korean',
 'Italian',
 'Pizza',
 'Chicken Wings',
 'Breakfast & Brunch',
 'Street Vendors',
 'Burgers',
 'Pizza',
 'Pizza',
 'Local Flavor',
 'Chinese',
 'Mexican',
 'Breakfast & Brunch',
 'Convenience Stores',
 'Mailbox Centers',
 'Vegetarian',
 'Seafood',
 'Sandwiches',
 'Mexican',
 'Ramen',
 'Filipino',
 'Pizza',
 'Vietnamese',
 'Street Vendors',
 'Ethical Grocery',
 'Desserts',
 'American (New)',
 'Mexican',
 'Asian Fusion',
 'Noodles',
 'Honduran',
 'Street Vend

In [74]:
df_b['category'] = category
df_b.sample(n=10)

,Unnamed: 0,business_id,name,address,categories,category
4202,4202,7P3Z4HTUM8M8Y9Yw3cc7Pg,Tipsy Coffee House,"6496 Medical Center St, Ste 102","Cafes, Desserts, Restaurants, Coffee & Tea, Fo...",Cafes
3167,3167,0pJiZDiKz1dfx5CIjce5XQ,Lui's Cocina,"1745 S Rainbow Blvd, Ste C","Restaurants, Mexican, Event Planning & Service...",Mexican
6004,6004,wRmPV-RvcqcjeyUeOaJQCg,The Bass Cafe Grill & Terrace,900 Karen Ave,"Salad, Food, Restaurants, Breakfast & Brunch, ...",Salad
6192,6192,cyzOijFJ82cae6qH6L8vuw,Arawan Thai Bistro and Dessert,"953 E Sahara Ave, Ste E-14","Food, Restaurants, Thai, Seafood, Desserts",Thai
4652,4652,UCpInuKIEWEOu4l2_Ejxsg,DaKao Quan,"5600 Spring Mountain Rd, Ste B","Vietnamese, Restaurants",Vietnamese
1355,1355,uuaH1UNcqcV-alT6nh42WA,Lino's Pizzeria & Italian Restaurant,4210 W Craig Rd,"Food Delivery Services, Restaurants, Pizza, Fo...",Pizza
2997,2997,G8SFOD9TKCSic2o0hyKc0Q,Ace Of Steaks,"5825 W Sahara Ave, Ste M","Restaurants, Steakhouses, Persian/Iranian, Kosher",Steakhouses
4478,4478,G7yeF8yMra1_kymdxPVCWQ,24 Hour Soul Food Express,4855 Boulder Hwy,"American (Traditional), Soul Food, Restaurants...",American (Traditional)
6304,6304,eiTWTVThkHr-DKXxIysbgA,Sweet PokÃ©,"7965 S Rainbow Blvd, Ste 135","Japanese, Asian Fusion, Hawaiian, Restaurants,...",Japanese
5163,5163,_8mBVtt6yhkS0YtT1FGBNg,Nekter Juice Bar,"1990 Village Center Cir, Ste 3","Food, Juice Bars & Smoothies, Vegan, Restauran...",Juice Bars & Smoothies


In [75]:
df_b['category'].unique()

array(['Italian', 'Mexican', 'Ice Cream & Frozen Yogurt', 'Chinese',
       'Pizza', 'French', 'Buffets', 'Bars', 'Sandwiches',
       'Middle Eastern', 'Dive Bars', 'Delis', 'American (Traditional)',
       'Steakhouses', 'Seafood', 'Bakeries', 'Coffee & Tea', 'Japanese',
       'Thai', 'Tapas/Small Plates', 'Southern', 'Korean',
       'Chicken Wings', 'Breakfast & Brunch', 'Street Vendors', 'Burgers',
       'Local Flavor', 'Convenience Stores', 'Mailbox Centers',
       'Vegetarian', 'Ramen', 'Filipino', 'Vietnamese', 'Ethical Grocery',
       'Desserts', 'American (New)', 'Asian Fusion', 'Noodles',
       'Honduran', 'Nightlife', 'Hotels & Travel', 'Tex-Mex', 'Ethiopian',
       'Shopping Centers', 'Soup', 'Sushi Bars', 'Gluten-Free', 'Beer',
       'Pubs', 'Barbeque', 'Poke', 'Hot Dogs', 'Caterers', 'Caribbean',
       'Active Life', 'Chicken Shop', 'Cafes', 'Hookah Bars',
       'Arts & Entertainment', 'Shopping', 'Hawaiian', 'Diners',
       'Waffles', 'Brazilian', 'Latin Ameri

In [88]:
df_b = df_b[df_b['category'].isin(['Italian', 'Mexican', 'Ice Cream & Frozen Yogurt', 'Chinese',
       'Pizza', 'French', 'Buffets', 'Bars', 'Sandwiches',
       'Middle Eastern', 'Dive Bars', 'Delis', 'American (Traditional)',
       'Steakhouses', 'Seafood', 'Bakeries', 'Coffee & Tea', 'Japanese',
       'Thai', 'Tapas/Small Plates', 'Southern', 'Korean',
       'Chicken Wings', 'Breakfast & Brunch', 'Street Vendors', 'Burgers',
       'Vegetarian', 'Ramen', 'Filipino', 'Vietnamese'
       'Desserts', 'American (New)', 'Asian Fusion', 'Noodles', 'Tex-Mex', 'Ethiopian', 'Soup', 'Sushi Bars', 'Gluten-Free', 'Beer',
       'Pubs', 'Barbeque', 'Poke', 'Hot Dogs',  'Caribbean','Chicken Shop', 'Cafes', 'Hookah Bars', 'Hawaiian', 'Diners',
       'Waffles', 'Brazilian', 'Latin American', 'Wine Bars',
       'Salad', 'Bagels', 'Argentine', 'Gastropubs', 
       'Lounges', 'Cheesesteaks', 'Tapas Bars', 'Grocery', 'Indian',
       'Butcher', 'Cajun/Creole', 'Modern European', 'Gelato', 'Donuts',
        'Dentists', 'Pasta Shops', 'Greek', 'Mongolian',
       'Bistros', 'Arabian', 'Tacos', 'Cocktail Bars', 
       'Mediterranean', 'Sports Bars',
       'Cantonese','Cafeteria', 'Wineries', 'Taiwanese',
       'Peruvian', 'Szechuan', 'Vegan', 'Coffee Roasteries',
        'Shaved Ice', 'Custom Cakes', 'Patisserie/Cake Shop', 'Spanish',
       'Fish & Chips', 'Tea Rooms', 'Cuban', 'Breweries', 'Shanghainese', 'Bubble Tea',
       'Themed Cafes', 'Teppanyaki', 'Venezuelan', 'Pretzels', 'Creperies', 'Pakistani',
       'Brasseries', 'Singaporean',
       'Japanese Curry', 'German', 'Afghan', 'New Mexican Cuisine',
       'Pan Asian', 'Gay Bars', 'Beer Bar', 'Hot Pot',
       'Seafood Markets', 'Persian/Iranian',
       'Irish', 'Conveyor Belt Sushi','Turkish',
       'Guamanian', 'Falafel', 'Ukrainian', 'Russian',
       'Laotian', 'Smokehouse', 'Puerto Rican',
       'Dinner Theater', 'Piano Bars', 'Beer Gardens','British', 'Lebanese',
        'African', 'Malaysian', 'South African',
       'Belgian', 'Kebab', 'Coffeeshops',
       'Colombian', 'Moroccan',
       'Acai Bowls'])]

In [89]:
len(df_b)

5954

In [90]:
df = df[df['business_id'].isin(df_b['business_id'])]

In [91]:
len(df)

1026019

In [92]:
df_temp = pd.merge(df, df_b[['business_id','category']], on='business_id')
df_temp['category'].unique()

array(['Delis', 'Breakfast & Brunch', 'Chicken Wings', 'Tapas Bars',
       'Wine Bars', 'Sandwiches', 'Cafes', 'American (New)',
       'Asian Fusion', 'Italian', 'Indian', 'Pubs', 'Bars', 'Seafood',
       'Diners', 'American (Traditional)', 'Chinese', 'Latin American',
       'Mexican', 'Thai', 'Korean', 'Tacos', 'Bagels', 'Argentine',
       'Burgers', 'Barbeque', 'Mediterranean', 'Sushi Bars', 'Soup',
       'Gastropubs', 'Cajun/Creole', 'Cantonese', 'Pizza', 'Buffets',
       'Vegetarian', 'Ramen', 'Greek', 'Japanese', 'Beer',
       'Ice Cream & Frozen Yogurt', 'French', 'Gluten-Free', 'Bakeries',
       'Arabian', 'Salad', 'Lounges', 'Gelato', 'Hawaiian', 'Steakhouses',
       'Pasta Shops', 'Sports Bars', 'Street Vendors', 'Noodles',
       'Coffee & Tea', 'Middle Eastern', 'Tapas/Small Plates', 'Tex-Mex',
       'Ethiopian', 'Filipino', 'Chicken Shop', 'Donuts', 'Brazilian',
       'Butcher', 'Cheesesteaks', 'Poke', 'Cocktail Bars', 'Wineries',
       'Caribbean', 'Waffles', 